# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [ ]:
!pip freeze | grep gensim

gensim==4.2.0


In [ ]:
!pip freeze | grep tensorflow

tensorflow==2.13.0rc1
tensorflow-addons==0.20.0
tensorflow-datasets==4.6.0
tensorflow-estimator==2.13.0rc0
tensorflow-macos==2.13.0rc1
tensorflow-metadata==1.10.0
tensorflow-metal==0.8.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [ ]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

Metal device set to: Apple M2 Pro


2023-11-24 16:14:59.657216: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-24 16:14:59.657233: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/layers_embedding.png" alt="Joint learning of embedding and RNN" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word2vec_representation.png" alt="Training of RNN and embedding independently" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [ ]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [ ]:
wv['dog']

array([-0.15840852,  0.15547955, -0.21424466,  0.21217652, -0.03408825,
       -0.33386117, -0.01700388,  0.58532315, -0.20426509, -0.24964602,
        0.01117653, -0.37483254, -0.06361631,  0.2210619 , -0.05175326,
       -0.28301215,  0.17365578, -0.28872487,  0.01279026, -0.29077125,
        0.185653  ,  0.06500498,  0.32139382, -0.13811941, -0.00607183,
        0.04154092, -0.22032873, -0.05683693, -0.13120273,  0.04212444,
        0.13890584, -0.0323867 ,  0.14320365, -0.39249888, -0.16395783,
        0.17345531,  0.15952027, -0.1727558 , -0.24395138, -0.26676577,
       -0.15409835, -0.32136738, -0.27861682,  0.17543775,  0.3356982 ,
       -0.03413855, -0.20273891, -0.13619095,  0.28288218,  0.20549928,
        0.07657087, -0.23573503, -0.24573867, -0.02678715, -0.25868338,
        0.21052684,  0.16743213, -0.07516544, -0.11329747,  0.00505033,
        0.10188274, -0.03105042,  0.01756213,  0.04550432, -0.18982583,
        0.2070637 , -0.05629909,  0.20795868, -0.2587135 ,  0.21

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [ ]:
size = len(wv['cat'])

print(f'The embedding space is of size {size}')

The embedding space is of size 100


🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [ ]:
wv.most_similar("car")

[('hit', 0.9750576615333557),
 ('escape', 0.9735110402107239),
 ('floor', 0.9715396165847778),
 ('crying', 0.9693775773048401),
 ('prison', 0.9688467383384705),
 ('wedding', 0.968353807926178),
 ('past', 0.9675046801567078),
 ('body', 0.9657547473907471),
 ('bed', 0.9648675322532654),
 ('grave', 0.9639755487442017)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [ ]:
word_embedding = wv['car']
wv.similar_by_vector(word_embedding)

[('car', 1.0),
 ('hit', 0.9750576615333557),
 ('escape', 0.9735110402107239),
 ('floor', 0.9715396165847778),
 ('crying', 0.9693775773048401),
 ('prison', 0.9688467383384705),
 ('wedding', 0.968353807926178),
 ('past', 0.9675046801567078),
 ('body', 0.9657547473907471),
 ('bed', 0.9648675322532654)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [ ]:
wv['good'] - wv['bad']

array([ 3.40838432e-02, -9.43998247e-02,  3.29184324e-01,  8.64837170e-02,
        2.19846368e-02, -2.45266318e-01, -2.18638659e-01,  1.25738710e-01,
       -2.14025974e-02, -5.39188743e-01,  1.97586253e-01, -8.71732235e-02,
        1.23539805e-01,  4.08248939e-02, -2.53344327e-02,  1.97439700e-01,
       -2.54235148e-01,  2.65944809e-01, -2.01002777e-01, -3.60777378e-02,
       -1.48113221e-01,  1.82885855e-01, -2.60850489e-01, -1.10828578e-01,
       -1.03871770e-01, -7.28017837e-02,  5.48168123e-02,  5.71420193e-01,
       -5.45804501e-02, -5.56780636e-01, -4.88710284e-01,  1.01565935e-01,
        2.98589885e-01, -8.01385045e-02, -3.28417182e-01, -2.12946773e-01,
        7.34921873e-01, -5.36821604e-01, -3.83452058e-01,  3.94809365e-01,
       -1.25887454e-01, -6.17489219e-01, -4.78436232e-01,  1.03760362e-02,
        4.48420107e-01, -1.63961306e-01,  5.21877229e-01, -1.77884668e-01,
       -2.41063714e-01,  1.41779318e-01,  8.88527185e-03,  1.25650942e-01,
       -4.73017752e-01, -

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [ ]:
res = wv['good'] - wv['bad'] + wv['stupid']

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [ ]:
wv.similar_by_vector(res)

[('nice', 0.7583796381950378),
 ('decent', 0.7577625513076782),
 ('potential', 0.7508993148803711),
 ('developed', 0.7296116948127747),
 ('vanity', 0.7286748290061951),
 ('hailed', 0.7283141016960144),
 ('such', 0.7226009964942932),
 ('good', 0.7194260954856873),
 ('baldwin', 0.7180870771408081),
 ('deed', 0.7176533341407776)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [ ]:
res = wv['queen'] - wv['king'] + wv['actor']
wv.similar_by_vector(res)

[('actor', 0.9805742502212524),
 ('actress', 0.8992334604263306),
 ('performance', 0.8824365139007568),
 ('guy', 0.8800014853477478),
 ('role', 0.8768371939659119),
 ('job', 0.8465615510940552),
 ('character', 0.8352804183959961),
 ('pet', 0.8190711140632629),
 ('billed', 0.8046131134033203),
 ('inexperienced', 0.8006188273429871)]

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [ ]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size=50)
wv2 = word2vec_2.wv
len(wv2['movie'])

50

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [ ]:
print('Vocabulary size', len(wv2.key_to_index))

diff_words = set([_ for elt in X_train for _ in elt])
print('Number of different words in the train set', len(diff_words))

Vocabulary size 8006
Number of different words in the train set 30419


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [ ]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=50, min_count=40)
word2vec_4 = Word2Vec(sentences=X_train, vector_size=50, min_count=2)

print(f'Number of word in W2V #1 : {len(wv.key_to_index)}')
print(f'Number of word in W2V #2 : {len(wv2.key_to_index)}')
print(f'Number of word in W2V #3 : {len(word2vec_3.wv.key_to_index)}')
print(f'Number of word in W2V #4 : {len(word2vec_4.wv.key_to_index)}')

Number of word in W2V #1 : 8006
Number of word in W2V #2 : 8006
Number of word in W2V #3 : 1385
Number of word in W2V #4 : 16729


Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [ ]:
word2vec_5 = Word2Vec(sentences=X_train, vector_size=50, min_count=40, window=10)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word2vec_representation.png" alt="Word2Vec and RNN" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [ ]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    # $CHALLENGIFY_BEGIN
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)
    # $CHALLENGIFY_END

### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [ ]:
def embedding(word2vec, sentences):
    # $CHALLENGIFY_BEGIN
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed
    # $CHALLENGIFY_END

X_train_embedded = embedding(word2vec, X_train)
X_test_embedded = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [ ]:
### YOUR CODE HERE
# $DELETE_BEGIN
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train_embedded, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_embedded, dtype='float32', padding='post')

# $DELETE_END
assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
